In [1]:
## Polecenia do instalacji pakietów
# conda install -c conda-forge tensorflow
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
data_AUG = pd.read_csv("apartments_pl_2023_08.csv")
data_SEP = pd.read_csv("apartments_pl_2023_09.csv")
data_OCT = pd.read_csv("apartments_pl_2023_10.csv")

# Analiza danych

Wyniki analizy danych (specyficzne dla konkretnego zbioru danych) standardowo powinny zawierać przynajmniej następujące elementy:
* liczba atrybutów i typy ich wartości
* liczba różnych wartości atrybutów, które nie są ciągłe (ze skończonego, niewielkiego zakresu)
* atrybut decyzyjny - charakterystyka
* brakujące wartości (z podziałem na obiekty i w ramach poszczególnych atrybutów warunkowych)
* duplikaty
* liczebności przykładów z poszczególnych klas
* współczynnik IR (imbalanced ratio)
* mapa korelacji atrybutów
* weryfikacja zasadności pozostawienia/usunięcia poszczególnych atrybutów na podstawie wiedzy dziedzinowej
* analiza (komentarze dodawane na bieżąco w notatniku)
* wnioski

In [3]:
print("data_AUG length: ", len(data_AUG))
print("data_SEP length: ", len(data_SEP))
print("data_OCT length: ", len(data_OCT))

data_AUG length:  18905
data_SEP length:  16997
data_OCT length:  16690


In [4]:
data_AUG.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,f8524536d4b09a0c8ccc0197ec9d7bde,szczecin,blockOfFlats,63.00,3.0,4.0,10.0,1980.0,53.378933,14.625296,...,0.413,condominium,concreteSlab,NaN,yes,yes,yes,no,yes,415000
1,accbe77d4b360fea9735f138a50608dd,szczecin,blockOfFlats,36.00,2.0,8.0,10.0,NaN,53.442692,14.559690,...,0.205,cooperative,concreteSlab,NaN,no,yes,yes,no,yes,395995
2,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
3,0a68cd14c44ec5140143ece75d739535,szczecin,tenement,87.60,3.0,2.0,3.0,NaN,53.435100,14.532900,...,0.087,condominium,brick,NaN,yes,yes,no,no,yes,640000
4,f66320e153c2441edc0fe293b54c8aeb,szczecin,blockOfFlats,66.00,3.0,1.0,3.0,NaN,53.410278,14.503611,...,0.514,condominium,NaN,NaN,no,no,no,no,no,759000


In [5]:
data_SEP.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,71de9eb8a92ac12d7c94315228b70cfb,szczecin,tenement,123.45,4.0,2.0,3.0,NaN,53.432586,14.535685,...,0.321,condominium,brick,NaN,no,yes,no,no,yes,799000
1,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.073,cooperative,NaN,NaN,no,no,yes,no,no,420000
2,3b8819c6c9ffd55adfddfd5bafcc844d,szczecin,tenement,50.00,2.0,NaN,1.0,NaN,53.364400,14.660900,...,0.399,condominium,brick,NaN,yes,no,no,no,no,350000
3,708ccd264180315a4cfe6324ed4861d0,szczecin,blockOfFlats,55.19,2.0,1.0,2.0,2013.0,53.480364,14.541494,...,0.183,condominium,brick,premium,yes,yes,no,yes,yes,607000
4,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000


In [6]:
data_OCT.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.073,cooperative,NaN,NaN,no,no,yes,no,no,420000
1,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
2,ede1c0d953ff85f3cd9676d39ca8f62d,szczecin,blockOfFlats,37.00,2.0,8.0,10.0,1970.0,53.442692,14.559690,...,0.205,cooperative,concreteSlab,NaN,no,yes,yes,yes,yes,429995
3,6c807c6c86a74c57a3fd4445a1f49a46,szczecin,apartmentBuilding,48.00,2.0,4.0,6.0,NaN,53.435461,14.570436,...,0.177,condominium,brick,NaN,no,yes,no,no,no,550000
4,bb14c9a540275da73f6cb6ed42e5c26c,szczecin,apartmentBuilding,55.19,2.0,1.0,2.0,2013.0,53.482094,14.542574,...,0.043,condominium,brick,premium,yes,yes,no,yes,yes,607000


In [7]:
# złączenie plików z danymi
data_ALL = pd.concat([data_AUG, data_SEP, data_OCT], ignore_index=True)

In [8]:
data_ALL.sample(10)

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
46958,2afe0f29ef91fd45a2c6bc534fc85733,warszawa,tenement,49.50,3.0,1.0,2.0,1910.0,52.349824,20.939341,...,0.249,condominium,brick,NaN,yes,yes,no,no,no,569250
2910,9701b09c87a59a1772ea2475b447332f,krakow,apartmentBuilding,56.31,3.0,3.0,3.0,2017.0,50.089836,19.872719,...,1.134,condominium,brick,premium,yes,yes,yes,no,no,949000
33726,c4da8bf1519a792dd3e461c0f47b676c,warszawa,blockOfFlats,78.50,3.0,3.0,4.0,1996.0,52.200600,20.871600,...,0.521,cooperative,brick,NaN,no,no,NaN,no,no,720000
47051,072424d4a25623d49d162b6d0d6e9740,warszawa,apartmentBuilding,37.92,2.0,7.0,7.0,2013.0,52.250026,21.070889,...,NaN,condominium,brick,premium,no,no,yes,no,no,568800
39976,c10ca67bd72ca1e4313d146cc98d8853,poznan,blockOfFlats,55.13,2.0,NaN,3.0,2023.0,52.446700,16.952400,...,0.198,condominium,NaN,NaN,no,yes,yes,no,no,500000
45896,47752253a2b8b9761e8ca8bc2f22fdea,katowice,tenement,43.00,2.0,4.0,4.0,1935.0,50.253100,19.016500,...,0.114,condominium,brick,NaN,no,no,no,no,no,365000
1227,8cbf186b7bcb3f39d94a3e1f527221cc,gdynia,NaN,38.00,2.0,5.0,10.0,1980.0,54.521388,18.539398,...,0.092,condominium,concreteSlab,low,no,no,no,no,yes,459000
11680,65fc0ace6ab0a6f9d577edd520ca6e79,lublin,blockOfFlats,48.00,3.0,2.0,4.0,1970.0,51.212225,22.571392,...,0.788,condominium,NaN,low,no,yes,no,no,yes,329000
34456,ee601f3330bfb14f0fc6514d710ab213,warszawa,blockOfFlats,79.40,3.0,9.0,9.0,1998.0,52.232444,20.988493,...,0.100,condominium,NaN,NaN,no,yes,yes,no,no,1750000
7526,2dc178861a2e3fa841f3a431f7e92245,wroclaw,apartmentBuilding,60.00,2.0,2.0,4.0,2008.0,51.060932,17.003532,...,0.157,condominium,brick,NaN,yes,yes,yes,no,no,630000


## Liczba atrybutów i typy ich wartości

In [9]:
data_SEP.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16997 entries, 0 to 16996
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    16997 non-null  object 
 1   city                  16997 non-null  object 
 2   type                  13171 non-null  object 
 3   squareMeters          16997 non-null  float64
 4   rooms                 16997 non-null  float64
 5   floor                 13954 non-null  float64
 6   floorCount            16751 non-null  float64
 7   buildYear             14048 non-null  float64
 8   latitude              16997 non-null  float64
 9   longitude             16997 non-null  float64
 10  centreDistance        16997 non-null  float64
 11  poiCount              16997 non-null  float64
 12  schoolDistance        16982 non-null  float64
 13  clinicDistance        16918 non-null  float64
 14  postOfficeDistance    16974 non-null  float64
 15  kindergartenDistanc

### Sierpień

In [10]:
data_AUG.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,18905.000000,18905.000000,15467.000000,18643.000000,15634.000000,18905.000000,18905.000000,18905.000000,18905.000000,18891.000000,18817.000000,18880.000000,18892.000000,18847.000000,18340.000000,18878.000000,1.890500e+04
mean,58.509976,2.676752,3.339820,5.279783,1984.551043,52.020411,19.516394,4.398444,20.099656,0.414897,1.002196,0.518748,0.371033,0.356646,1.460047,0.364134,6.925592e+05
std,21.060168,0.903619,2.549141,3.333666,33.767929,1.353998,1.769053,2.857387,23.513874,0.472782,0.913721,0.506783,0.452338,0.475884,1.102847,0.469334,3.443890e+05
min,25.000000,1.000000,1.000000,1.000000,1850.000000,49.979110,14.447127,0.020000,0.000000,0.004000,0.005000,0.001000,0.004000,0.001000,0.006000,0.003000,1.500000e+05
25%,44.200000,2.000000,2.000000,3.000000,1967.000000,51.109100,18.536220,2.070000,7.000000,0.175000,0.367000,0.237000,0.158000,0.118500,0.592000,0.145000,4.588000e+05
50%,54.400000,3.000000,3.000000,4.000000,1990.000000,52.193604,19.908944,4.010000,13.000000,0.288000,0.700000,0.387000,0.264000,0.235000,1.140000,0.239000,6.200000e+05
75%,68.200000,3.000000,4.000000,6.000000,2014.000000,52.412561,20.992010,6.180000,23.000000,0.466500,1.286000,0.621000,0.415000,0.417000,2.107000,0.405000,8.300000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.583210,23.207128,16.940000,208.000000,4.818000,4.996000,4.967000,4.960000,4.985000,4.998000,4.992000,2.500000e+06


### Wrzesień

In [11]:
data_SEP.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,16997.000000,16997.000000,13954.000000,16751.000000,14048.000000,16997.000000,16997.000000,16997.000000,16997.000000,16982.000000,16918.000000,16974.000000,16983.000000,16954.000000,16495.000000,16974.000000,1.699700e+04
mean,59.033912,2.689357,3.327576,5.237538,1984.351936,52.066227,19.528680,4.347108,20.345590,0.418712,0.988139,0.524299,0.378864,0.359300,1.451509,0.370333,7.080972e+05
std,21.362743,0.916103,2.532150,3.305643,34.041196,1.331268,1.790088,2.893583,23.710083,0.482618,0.911026,0.516412,0.467429,0.487905,1.109576,0.483060,3.519888e+05
min,25.000000,1.000000,1.000000,1.000000,1851.000000,49.979110,14.447127,0.020000,0.000000,0.004000,0.005000,0.003000,0.002000,0.001000,0.007000,0.001000,1.549000e+05
25%,44.500000,2.000000,2.000000,3.000000,1965.000000,51.120355,18.529520,1.960000,6.000000,0.175000,0.359000,0.237000,0.158000,0.116000,0.583000,0.146000,4.650000e+05
50%,55.190000,3.000000,3.000000,4.000000,1991.000000,52.197569,19.903611,3.930000,13.000000,0.292000,0.681000,0.391000,0.268000,0.234000,1.122000,0.240000,6.390000e+05
75%,69.000000,3.000000,4.000000,6.000000,2014.000000,52.450200,20.998580,6.150000,24.000000,0.469000,1.266000,0.629000,0.423000,0.418000,2.077500,0.411000,8.500000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.606460,23.207128,16.920000,208.000000,4.818000,4.989000,4.905000,4.960000,4.985000,5.000000,4.861000,2.500000e+06


### Październik

In [12]:
data_OCT.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,16690.000000,16690.000000,13644.000000,16448.000000,13785.000000,16690.000000,16690.000000,16690.000000,16690.000000,16669.000000,16605.000000,16665.000000,16668.000000,16637.000000,16185.000000,16661.000000,1.669000e+04
mean,58.985279,2.680228,3.268030,5.162755,1984.573957,52.047413,19.489039,4.328341,20.462912,0.417843,0.978049,0.526101,0.376593,0.357309,1.444789,0.365081,7.235462e+05
std,21.347723,0.911191,2.455211,3.201862,34.164553,1.331076,1.779749,2.879183,24.104101,0.480638,0.905555,0.515941,0.463518,0.483880,1.105216,0.476461,3.521544e+05
min,25.000000,1.000000,1.000000,1.000000,1851.000000,49.981357,14.447127,0.020000,0.000000,0.005000,0.001000,0.003000,0.002000,0.001000,0.007000,0.001000,1.526200e+05
25%,44.402500,2.000000,2.000000,3.000000,1965.000000,51.114060,18.512167,1.970000,7.000000,0.175000,0.355000,0.241000,0.157000,0.117000,0.586000,0.145000,4.800000e+05
50%,55.000000,3.000000,3.000000,4.000000,1992.000000,52.195677,19.894169,3.920000,13.000000,0.292000,0.674000,0.395000,0.266000,0.235000,1.119000,0.242000,6.590000e+05
75%,69.000000,3.000000,4.000000,6.000000,2015.000000,52.440733,20.994512,6.090000,24.000000,0.468000,1.246000,0.629000,0.423000,0.419000,2.051000,0.407000,8.700000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.606460,23.195701,16.920000,208.000000,4.818000,4.989000,4.905000,4.960000,4.985000,5.000000,4.861000,2.500000e+06


### Dane złączone

In [13]:
data_ALL.describe()

,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,price
count,52592.000000,52592.000000,43065.000000,51842.000000,43467.000000,52592.000000,52592.000000,52592.000000,52592.000000,52542.000000,52340.000000,52519.000000,52543.000000,52438.000000,51020.000000,52513.000000,5.259200e+04
mean,58.830142,2.681929,3.313108,5.229004,1984.493961,52.043787,19.511684,4.359606,20.294417,0.417065,0.989992,0.522876,0.375328,0.357714,1.452446,0.366438,7.074146e+05
std,21.250642,0.910070,2.514338,3.283571,33.981805,1.339533,1.779307,2.876144,23.765901,0.478467,0.910303,0.512821,0.460811,0.482326,1.105774,0.476065,3.495542e+05
min,25.000000,1.000000,1.000000,1.000000,1850.000000,49.979110,14.447127,0.020000,0.000000,0.004000,0.001000,0.001000,0.002000,0.001000,0.006000,0.001000,1.500000e+05
25%,44.360000,2.000000,2.000000,3.000000,1965.000000,51.114060,18.527700,2.010000,7.000000,0.175000,0.360000,0.238000,0.158000,0.117000,0.587000,0.145000,4.680000e+05
50%,55.000000,3.000000,3.000000,4.000000,1991.000000,52.195252,19.902880,3.960000,13.000000,0.290000,0.685000,0.390000,0.266000,0.235000,1.124000,0.240000,6.390000e+05
75%,69.000000,3.000000,4.000000,6.000000,2014.000000,52.436800,20.994660,6.140000,24.000000,0.468000,1.270000,0.628000,0.420000,0.418000,2.076000,0.407000,8.500000e+05
max,150.000000,6.000000,29.000000,29.000000,2023.000000,54.606460,23.207128,16.940000,208.000000,4.818000,4.996000,4.967000,4.960000,4.985000,5.000000,4.992000,2.500000e+06


## Liczba różnych wartości atrybutów, które nie są ciągłe (ze skończonego, niewielkiego zakresu)

In [14]:
def get_unique_count(data_frame):
    temp = []
    for column in data_frame.columns.values:
        temp.append([column, len(data_frame[column].unique())])
        
    data_SEP_unique = pd.DataFrame(np.array(temp), columns=["attribute", "unique"])
    return data_SEP_unique

In [15]:
print("Liczba unikalnych wartości dla poszczególnych kolumn - Sierpień")
get_unique_count(data_AUG)

Liczba unikalnych wartości dla poszczególnych kolumn - Sierpień


,attribute,unique
0,id,18905
1,city,15
2,type,4
3,squareMeters,4214
4,rooms,6
5,floor,25
6,floorCount,30
7,buildYear,156
8,latitude,13762
9,longitude,14016


In [16]:
print("Liczba unikalnych wartości dla poszczególnych kolumn - Wrzesień")
get_unique_count(data_SEP)

Liczba unikalnych wartości dla poszczególnych kolumn - Wrzesień


,attribute,unique
0,id,16997
1,city,15
2,type,4
3,squareMeters,4042
4,rooms,6
5,floor,24
6,floorCount,30
7,buildYear,154
8,latitude,12493
9,longitude,12695


In [17]:
print("Liczba unikalnych wartości dla poszczególnych kolumn - Październik")
get_unique_count(data_OCT)

Liczba unikalnych wartości dla poszczególnych kolumn - Październik


,attribute,unique
0,id,16690
1,city,15
2,type,4
3,squareMeters,4068
4,rooms,6
5,floor,24
6,floorCount,30
7,buildYear,152
8,latitude,12340
9,longitude,12513


In [18]:
print("Liczba unikalnych wartości dla poszczególnych kolumn - wszystkie dane")
get_unique_count(data_ALL)

Liczba unikalnych wartości dla poszczególnych kolumn - wszystkie dane


,attribute,unique
0,id,32027
1,city,15
2,type,4
3,squareMeters,5175
4,rooms,6
5,floor,25
6,floorCount,30
7,buildYear,158
8,latitude,20723
9,longitude,21215


In [19]:
# TODO: prawdopodobnie wszystko co ma liczność powyżej setek będzie tutaj do wycięcia, 
# bo mają być nieciągłe atrybuty. Będzie można po prostu zrobić tablicę z nazwami kolumn i 
# wyświetlić tylko te, które są potrzebne

## Atrybut decyzyjny - charakterystyka

In [20]:
price_AUG = data_AUG["price"].rename("price_AUG")
price_SEP = data_SEP["price"].rename("price_SEP")
price_OCT = data_OCT["price"].rename("price_OCT")

In [21]:
# porównanie statystyk cen z poszczególnych miesięcy
pd.concat([
        price_AUG.describe().map(int), 
        price_SEP.describe().map(int), 
        price_OCT.describe().map(int)
    ],
    axis=1
)

,price_AUG,price_SEP,price_OCT
count,18905,16997,16690
mean,692559,708097,723546
std,344389,351988,352154
min,150000,154900,152620
25%,458800,465000,480000
50%,620000,639000,659000
75%,830000,850000,870000
max,2500000,2500000,2500000


## Brakujące wartości (z podziałem na obiekty i w ramach poszczególnych atrybutów warunkowych)

In [64]:
# ilość wierszy z brakujacymi danymi
# ilość braków poszczególnych kolumnach

In [23]:
def print_missing_values(data_frame):
    for column in data_frame.columns.values:
        print("\t", data_frame[column].isna().sum(), "\t", column)

In [24]:
print("N/A values for each column (August): ")
print_missing_values(data_AUG)

N/A values for each column (August): 
	 0 	 id
	 0 	 city
	 4039 	 type
	 0 	 squareMeters
	 0 	 rooms
	 3438 	 floor
	 262 	 floorCount
	 3271 	 buildYear
	 0 	 latitude
	 0 	 longitude
	 0 	 centreDistance
	 0 	 poiCount
	 14 	 schoolDistance
	 88 	 clinicDistance
	 25 	 postOfficeDistance
	 13 	 kindergartenDistance
	 58 	 restaurantDistance
	 565 	 collegeDistance
	 27 	 pharmacyDistance
	 0 	 ownership
	 7387 	 buildingMaterial
	 14344 	 condition
	 0 	 hasParkingSpace
	 0 	 hasBalcony
	 926 	 hasElevator
	 0 	 hasSecurity
	 0 	 hasStorageRoom
	 0 	 price


In [25]:
print("N/A values for each column (September): ")
print_missing_values(data_SEP)

N/A values for each column (September): 
	 0 	 id
	 0 	 city
	 3826 	 type
	 0 	 squareMeters
	 0 	 rooms
	 3043 	 floor
	 246 	 floorCount
	 2949 	 buildYear
	 0 	 latitude
	 0 	 longitude
	 0 	 centreDistance
	 0 	 poiCount
	 15 	 schoolDistance
	 79 	 clinicDistance
	 23 	 postOfficeDistance
	 14 	 kindergartenDistance
	 43 	 restaurantDistance
	 502 	 collegeDistance
	 23 	 pharmacyDistance
	 0 	 ownership
	 6635 	 buildingMaterial
	 12958 	 condition
	 0 	 hasParkingSpace
	 0 	 hasBalcony
	 868 	 hasElevator
	 0 	 hasSecurity
	 0 	 hasStorageRoom
	 0 	 price


In [26]:
print("N/A values for each column (October): ")
print_missing_values(data_OCT)

N/A values for each column (October): 
	 0 	 id
	 0 	 city
	 3785 	 type
	 0 	 squareMeters
	 0 	 rooms
	 3046 	 floor
	 242 	 floorCount
	 2905 	 buildYear
	 0 	 latitude
	 0 	 longitude
	 0 	 centreDistance
	 0 	 poiCount
	 21 	 schoolDistance
	 85 	 clinicDistance
	 25 	 postOfficeDistance
	 22 	 kindergartenDistance
	 53 	 restaurantDistance
	 505 	 collegeDistance
	 29 	 pharmacyDistance
	 0 	 ownership
	 6483 	 buildingMaterial
	 12517 	 condition
	 0 	 hasParkingSpace
	 0 	 hasBalcony
	 894 	 hasElevator
	 0 	 hasSecurity
	 0 	 hasStorageRoom
	 0 	 price


In [63]:
print("N/A values for each column: ")
for column in data_ALL.columns.values:
    print("\t", data_ALL[column].isna().sum(), "\t", column)

N/A values for each column: 
	 0 	 id
	 0 	 city
	 11650 	 type
	 0 	 squareMeters
	 0 	 rooms
	 9527 	 floor
	 750 	 floorCount
	 9125 	 buildYear
	 0 	 latitude
	 0 	 longitude
	 0 	 centreDistance
	 0 	 poiCount
	 50 	 schoolDistance
	 252 	 clinicDistance
	 73 	 postOfficeDistance
	 49 	 kindergartenDistance
	 154 	 restaurantDistance
	 1572 	 collegeDistance
	 79 	 pharmacyDistance
	 0 	 ownership
	 20505 	 buildingMaterial
	 39819 	 condition
	 0 	 hasParkingSpace
	 0 	 hasBalcony
	 2688 	 hasElevator
	 0 	 hasSecurity
	 0 	 hasStorageRoom
	 0 	 price


## Duplikaty

### Duplikaty w danych z miesiąca sierpień

In [28]:
data_AUG.loc[data_ALL.duplicated()]

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price


### Duplikaty w danych z miesiąca wrzesień

In [29]:
data_SEP.loc[data_ALL.duplicated()]

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price


### Duplikaty w danych z miesiąca październik

In [30]:
data_OCT.loc[data_ALL.duplicated()]

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price


### Duplikaty w danych połączonych

In [31]:
duplicates_ALL = data_ALL.loc[data_ALL.duplicated()]
duplicates_ALL.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
18905,71de9eb8a92ac12d7c94315228b70cfb,szczecin,tenement,123.45,4.0,2.0,3.0,NaN,53.432586,14.535685,...,0.321,condominium,brick,NaN,no,yes,no,no,yes,799000
18906,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.073,cooperative,NaN,NaN,no,no,yes,no,no,420000
18909,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
18912,eb15c0a8a7f16a523f5cf5f61de06638,szczecin,blockOfFlats,49.50,2.0,1.0,4.0,1980.0,53.365833,14.661389,...,0.236,cooperative,NaN,low,no,yes,no,no,yes,462000
18922,209a6d2dea99216bdf83a968343cd3b3,szczecin,tenement,61.48,2.0,NaN,3.0,1900.0,53.491508,14.606985,...,1.521,condominium,brick,premium,no,no,no,no,yes,390000


In [32]:
len(duplicates_ALL)

17360

In [54]:
aug_test = data_AUG.drop(columns=["id"])
len(aug_test.loc[aug_test.duplicated()])

200

In [56]:
aug_test2 = data_AUG.drop(columns=["id", "price"])
len(aug_test2.loc[aug_test2.duplicated()])

241

In [60]:
aug_test3 = data_AUG.drop(columns=["price"])
len(aug_test3.loc[aug_test3.duplicated()])

0

In [93]:
data_AUG.loc[aug_test.duplicated()].head(100)

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
68,4e679733445ed71e835415cc1501ea39,szczecin,tenement,101.00,3.0,3.0,4.0,1930.0,53.431078,14.532763,...,0.186,condominium,brick,premium,no,yes,no,no,yes,559000
211,0457321a89c6f6413886f5095a5be635,szczecin,NaN,48.16,2.0,3.0,4.0,NaN,53.424180,14.548586,...,0.126,condominium,brick,NaN,no,no,no,no,no,339000
213,58b0d9314ca0f06298f2e69571f57e36,szczecin,NaN,31.00,1.0,NaN,4.0,1986.0,53.395814,14.525907,...,0.264,cooperative,concreteSlab,NaN,no,yes,no,no,no,285000
291,5e55fff04ee7488dc2c3d6d85cd7ab36,szczecin,blockOfFlats,76.00,4.0,NaN,2.0,2023.0,53.428544,14.552812,...,0.085,condominium,NaN,NaN,yes,yes,no,no,no,480000
369,eb9e63d8bc6dc1db2d436fe7d06d324c,szczecin,NaN,53.00,2.0,3.0,4.0,1930.0,53.445704,14.522719,...,0.499,condominium,brick,premium,no,yes,no,no,no,499000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6525,c82234263e58f5980b0ff5880a29c649,gdansk,tenement,42.40,2.0,NaN,2.0,1906.0,54.331190,18.633070,...,0.250,condominium,brick,NaN,no,no,no,no,no,319000
6549,3b1350de131a63d06753742112930518,gdansk,blockOfFlats,60.30,3.0,4.0,4.0,1984.0,54.338760,18.609750,...,0.232,condominium,concreteSlab,NaN,no,yes,no,no,yes,649000
6602,32d2461b088b94fda7315126fa726a46,gdansk,blockOfFlats,45.00,2.0,4.0,4.0,1970.0,54.415540,18.598900,...,0.485,condominium,NaN,NaN,no,yes,no,no,yes,485000
6622,53bc1a9a1de86dea7462db3073eda124,gdansk,NaN,57.93,3.0,3.0,4.0,2022.0,54.331320,18.550890,...,0.404,condominium,NaN,NaN,no,yes,yes,no,no,491500


In [94]:
display(data_AUG.loc[aug_test.duplicated()])

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
68,4e679733445ed71e835415cc1501ea39,szczecin,tenement,101.00,3.0,3.0,4.0,1930.0,53.431078,14.532763,...,0.186,condominium,brick,premium,no,yes,no,no,yes,559000
211,0457321a89c6f6413886f5095a5be635,szczecin,NaN,48.16,2.0,3.0,4.0,NaN,53.424180,14.548586,...,0.126,condominium,brick,NaN,no,no,no,no,no,339000
213,58b0d9314ca0f06298f2e69571f57e36,szczecin,NaN,31.00,1.0,NaN,4.0,1986.0,53.395814,14.525907,...,0.264,cooperative,concreteSlab,NaN,no,yes,no,no,no,285000
291,5e55fff04ee7488dc2c3d6d85cd7ab36,szczecin,blockOfFlats,76.00,4.0,NaN,2.0,2023.0,53.428544,14.552812,...,0.085,condominium,NaN,NaN,yes,yes,no,no,no,480000
369,eb9e63d8bc6dc1db2d436fe7d06d324c,szczecin,NaN,53.00,2.0,3.0,4.0,1930.0,53.445704,14.522719,...,0.499,condominium,brick,premium,no,yes,no,no,no,499000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18262,c6210f05ca54f39dca03587347f9d46f,bydgoszcz,blockOfFlats,42.00,2.0,NaN,11.0,NaN,53.124984,18.049752,...,0.155,cooperative,concreteSlab,low,no,yes,no,no,yes,274000
18349,a8a7747c101f69b259c502ff804dbacf,bydgoszcz,blockOfFlats,35.14,2.0,4.0,4.0,NaN,53.108100,18.033900,...,0.146,cooperative,concreteSlab,NaN,no,yes,no,no,yes,255000
18429,8e2c5d9d9453df942201faab3d5fdda4,bydgoszcz,blockOfFlats,31.00,1.0,NaN,11.0,NaN,53.124984,18.049752,...,0.155,condominium,concreteSlab,NaN,no,no,yes,no,yes,265000
18533,8daf39bb227e5bf222d2897ba0e012e9,bydgoszcz,blockOfFlats,54.00,3.0,1.0,3.0,2022.0,53.123344,18.007987,...,0.389,condominium,NaN,premium,no,yes,yes,no,no,485900


In [89]:
id_list = data_AUG.loc[aug_test.duplicated()].index.values.tolist()

In [90]:
data_AUG.loc[aug_test2.duplicated()].drop(index=id_list)

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
261,a766fe7cac8e590474f6886406d1a113,szczecin,NaN,68.20,3.0,1.0,1.0,NaN,53.477000,14.526000,...,1.239,condominium,NaN,NaN,no,no,no,no,no,590000
277,8eaa13f823df45ea8c05fc988da8dede,szczecin,NaN,60.64,3.0,NaN,4.0,1980.0,53.426960,14.537907,...,0.036,condominium,concreteSlab,premium,no,yes,no,no,no,429000
493,531963434cdda4a34d941cc51b1e42de,szczecin,tenement,70.00,3.0,1.0,3.0,1936.0,53.427926,14.540466,...,0.149,condominium,brick,low,no,no,no,no,yes,395000
510,fed5540a2eed16acd1d4f9ebda6db993,szczecin,blockOfFlats,76.64,3.0,2.0,4.0,1990.0,53.426320,14.531440,...,0.048,condominium,brick,premium,no,yes,no,no,yes,749000
778,3f66f710bf46cf4212ae9f302cc714af,gdynia,apartmentBuilding,54.10,3.0,3.0,3.0,2019.0,54.502670,18.541950,...,0.076,condominium,brick,NaN,no,yes,yes,no,no,990000
866,26fc0fb6ef68d058499b133b9f383dc9,gdynia,NaN,51.26,2.0,1.0,2.0,1952.0,54.541420,18.545440,...,1.280,condominium,brick,NaN,no,no,no,no,yes,325000
1733,d82995f7c1c1ebcf6498ddd3fbda5d76,krakow,blockOfFlats,49.00,3.0,3.0,4.0,1987.0,50.016528,19.948098,...,0.221,cooperative,concreteSlab,NaN,no,yes,no,no,yes,555000
2085,7b79293f7f07fce459774fe2652fc03c,krakow,tenement,45.80,2.0,3.0,4.0,1910.0,50.039000,19.941600,...,0.163,condominium,brick,NaN,yes,no,no,no,yes,299000
2797,d38997378b3f4a6667d6a2ede95d54e9,krakow,blockOfFlats,28.10,1.0,NaN,NaN,2022.0,50.067612,19.846828,...,1.924,condominium,NaN,NaN,no,no,NaN,no,no,344000
2955,fdfbb1c7ed51b4f71fe34f9b7c2d083e,krakow,blockOfFlats,32.87,2.0,NaN,5.0,2022.0,50.067330,20.008590,...,0.821,condominium,NaN,NaN,no,no,no,no,no,582000


In [57]:
sep_test = data_SEP.drop(columns=["id"])
len(sep_test.loc[sep_test.duplicated()])

175

In [58]:
sep_test2 = data_SEP.drop(columns=["id", "price"])
len(sep_test2.loc[sep_test2.duplicated()])

206

In [61]:
sep_test3 = data_SEP.drop(columns=["price"])
len(sep_test3.loc[sep_test3.duplicated()])

0

In [53]:
oct_test = data_OCT.drop(columns=["id"])
len(oct_test.loc[oct_test.duplicated()])

152

In [59]:
oct_test2 = data_OCT.drop(columns=["id", "price"])
len(oct_test2.loc[oct_test2.duplicated()])

173

In [62]:
oct_test3 = data_OCT.drop(columns=["price"])
len(oct_test3.loc[oct_test3.duplicated()])

0

In [48]:
test = data_ALL.drop(columns=["price"])
test.head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,collegeDistance,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom
0,f8524536d4b09a0c8ccc0197ec9d7bde,szczecin,blockOfFlats,63.00,3.0,4.0,10.0,1980.0,53.378933,14.625296,...,NaN,0.413,condominium,concreteSlab,NaN,yes,yes,yes,no,yes
1,accbe77d4b360fea9735f138a50608dd,szczecin,blockOfFlats,36.00,2.0,8.0,10.0,NaN,53.442692,14.559690,...,1.404,0.205,cooperative,concreteSlab,NaN,no,yes,yes,no,yes
2,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,1.857,0.280,condominium,brick,NaN,no,no,no,no,no
3,0a68cd14c44ec5140143ece75d739535,szczecin,tenement,87.60,3.0,2.0,3.0,NaN,53.435100,14.532900,...,0.310,0.087,condominium,brick,NaN,yes,yes,no,no,yes
4,f66320e153c2441edc0fe293b54c8aeb,szczecin,blockOfFlats,66.00,3.0,1.0,3.0,NaN,53.410278,14.503611,...,2.138,0.514,condominium,NaN,NaN,no,no,no,no,no


In [37]:
test.loc[test.duplicated()].head()

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,...,collegeDistance,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom
18905,71de9eb8a92ac12d7c94315228b70cfb,szczecin,tenement,123.45,4.0,2.0,3.0,NaN,53.432586,14.535685,...,0.388,0.321,condominium,brick,NaN,no,yes,no,no,yes
18906,23d558163bb1c7863c73485048e50858,szczecin,NaN,53.80,2.0,5.0,5.0,NaN,53.429429,14.554642,...,0.465,0.073,cooperative,NaN,NaN,no,no,yes,no,no
18909,8373aa373dbc3fe7ca3b7434166b8766,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,...,1.857,0.280,condominium,brick,NaN,no,no,no,no,no
18912,eb15c0a8a7f16a523f5cf5f61de06638,szczecin,blockOfFlats,49.50,2.0,1.0,4.0,1980.0,53.365833,14.661389,...,NaN,0.236,cooperative,NaN,low,no,yes,no,no,yes
18922,209a6d2dea99216bdf83a968343cd3b3,szczecin,tenement,61.48,2.0,NaN,3.0,1900.0,53.491508,14.606985,...,3.720,1.521,condominium,brick,premium,no,no,no,no,yes


In [39]:
len(test.loc[test.duplicated()])

19723

In [47]:
test2 = data_ALL.drop(columns=["id"])
test2.head()

,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
0,szczecin,blockOfFlats,63.00,3.0,4.0,10.0,1980.0,53.378933,14.625296,6.53,...,0.413,condominium,concreteSlab,NaN,yes,yes,yes,no,yes,415000
1,szczecin,blockOfFlats,36.00,2.0,8.0,10.0,NaN,53.442692,14.559690,2.15,...,0.205,cooperative,concreteSlab,NaN,no,yes,yes,no,yes,395995
2,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,3.24,...,0.280,condominium,brick,NaN,no,no,no,no,no,565000
3,szczecin,tenement,87.60,3.0,2.0,3.0,NaN,53.435100,14.532900,2.27,...,0.087,condominium,brick,NaN,yes,yes,no,no,yes,640000
4,szczecin,blockOfFlats,66.00,3.0,1.0,3.0,NaN,53.410278,14.503611,4.07,...,0.514,condominium,NaN,NaN,no,no,no,no,no,759000


In [41]:
test2.loc[test2.duplicated()].head()

,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,...,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
68,szczecin,tenement,101.00,3.0,3.0,4.0,1930.0,53.431078,14.532763,2.05,...,0.186,condominium,brick,premium,no,yes,no,no,yes,559000
211,szczecin,NaN,48.16,2.0,3.0,4.0,NaN,53.424180,14.548586,0.82,...,0.126,condominium,brick,NaN,no,no,no,no,no,339000
213,szczecin,NaN,31.00,1.0,NaN,4.0,1986.0,53.395814,14.525907,3.85,...,0.264,cooperative,concreteSlab,NaN,no,yes,no,no,no,285000
291,szczecin,blockOfFlats,76.00,4.0,NaN,2.0,2023.0,53.428544,14.552812,0.79,...,0.085,condominium,NaN,NaN,yes,yes,no,no,no,480000
369,szczecin,NaN,53.00,2.0,3.0,4.0,1930.0,53.445704,14.522719,3.55,...,0.499,condominium,brick,premium,no,yes,no,no,no,499000


In [42]:
len(test2.loc[test2.duplicated()])

18632

In [46]:
test3 = data_ALL.drop(columns=["id", "price"])
test3.head()

,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,...,collegeDistance,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom
0,szczecin,blockOfFlats,63.00,3.0,4.0,10.0,1980.0,53.378933,14.625296,6.53,...,NaN,0.413,condominium,concreteSlab,NaN,yes,yes,yes,no,yes
1,szczecin,blockOfFlats,36.00,2.0,8.0,10.0,NaN,53.442692,14.559690,2.15,...,1.404,0.205,cooperative,concreteSlab,NaN,no,yes,yes,no,yes
2,szczecin,tenement,73.02,3.0,2.0,3.0,NaN,53.452222,14.553333,3.24,...,1.857,0.280,condominium,brick,NaN,no,no,no,no,no
3,szczecin,tenement,87.60,3.0,2.0,3.0,NaN,53.435100,14.532900,2.27,...,0.310,0.087,condominium,brick,NaN,yes,yes,no,no,yes
4,szczecin,blockOfFlats,66.00,3.0,1.0,3.0,NaN,53.410278,14.503611,4.07,...,2.138,0.514,condominium,NaN,NaN,no,no,no,no,no


In [44]:
test3.loc[test3.duplicated()].head()

,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,...,collegeDistance,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom
68,szczecin,tenement,101.00,3.0,3.0,4.0,1930.0,53.431078,14.532763,2.05,...,0.644,0.186,condominium,brick,premium,no,yes,no,no,yes
211,szczecin,NaN,48.16,2.0,3.0,4.0,NaN,53.424180,14.548586,0.82,...,0.672,0.126,condominium,brick,NaN,no,no,no,no,no
213,szczecin,NaN,31.00,1.0,NaN,4.0,1986.0,53.395814,14.525907,3.85,...,2.698,0.264,cooperative,concreteSlab,NaN,no,yes,no,no,no
261,szczecin,NaN,68.20,3.0,1.0,1.0,NaN,53.477000,14.526000,6.39,...,3.819,1.239,condominium,NaN,NaN,no,no,no,no,no
277,szczecin,NaN,60.64,3.0,NaN,4.0,1980.0,53.426960,14.537907,1.58,...,0.459,0.036,condominium,concreteSlab,premium,no,yes,no,no,no


In [45]:
len(test3.loc[test3.duplicated()])

21496

Duplikaty występują jedynie po złączeniu danych z poszczególnych miesięcy. Może to ozanczać, że poszczególne oferty zostały wystawione kilkukrotnie w kolejnych miesiącach lub oferty te były aktywne na przestrzeni kilku miesięcy.

## Liczebności przykładów z poszczególnych klas

In [33]:
# TODO: Co w tym miejscu opisać dla dyskretyzacji

In [34]:
data_SEP["price"].value_counts()

599000     255
699000     238
799000     186
499000     183
650000     163
          ... 
384899       1
894374       1
619999       1
1284000      1
299900       1
Name: price, Length: 2555, dtype: int64

## Współczynnik IR (imbalanced ratio)

## Mapa korelacji atrybutów

## Weryfikacja zasadności pozostawienia/usunięcia poszczególnych atrybutów na podstawie wiedzy dziedzinowej

## Wnioski

# Przygotowanie danych

## Przypisanie miesiąca do poszczególnych ofert na podstawie pliku, z którego pochodzą dane

## Usunięcie duplikatów